In [1]:
# Install dependencies
!pip install faker pandas numpy graphviz streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.5 MB/s eta 0:00:00


In [2]:
# Create folder structure
!mkdir -p /content/pxt-hr-data-governance/{data/raw,data/staging,data/curated,metadata,src,notebooks,docs,dashboard}

In [3]:
# Generate synthetic HR employee data
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()
Faker.seed(42)
np.random.seed(42)

# Number of synthetic employees
n = 200

departments = ["HR", "Human Resources", "H.R.", "IT", "IT Dept", "Finance", "Operations", "Marketing", "Sales"]
job_titles = ["Analyst", "Manager", "Associate", "Director", "Engineer", "Coordinator"]

data = {
    "Employee_ID": [fake.unique.random_int(10000, 99999) for _ in range(n)],
    "First_Name": [fake.first_name() for _ in range(n)],
    "Last_Name": [fake.last_name() for _ in range(n)],
    "Department": np.random.choice(departments, n),
    "Job_Title": np.random.choice(job_titles, n),
    "Manager_ID": np.where(
        np.random.rand(n) > 0.2,  # 80% have managers
        [fake.random_int(10000, 99999) for _ in range(n)],
        None
    ),
    "Location": np.random.choice(["New York", "Wilmington", "Chicago", "Columbus", "Plano"], n),
    "Country": "USA",
    "Employment_Type": np.random.choice(["FT", "PT", "Contract"], n),
    "Hire_Date": [fake.date_between(start_date='-10y', end_date='today') for _ in range(n)],
    "Termination_Date": [
        fake.date_between(start_date='-5y', end_date='today') if np.random.rand() > 0.85 else None
        for _ in range(n)
    ],
    "Status": np.where(np.random.rand(n) > 0.85, "Terminated", "Active"),
    "Base_Salary": np.random.randint(45000, 150000, n)
}

df = pd.DataFrame(data)

# Intentionally add data quality problems
df = pd.concat([df, df.sample(5, random_state=1)], ignore_index=True)   # duplicates
df.loc[np.random.choice(df.index, 3), "Hire_Date"] = "2035-01-01"       # future hire dates
df.loc[np.random.choice(df.index, 2), "Department"] = "Hr"              # inconsistent departments

path = "/content/pxt-hr-data-governance/data/raw/hr_employees_raw.csv"
df.to_csv(path, index=False)
print("Synthetic HR data saved to:", path)
df.head()

Synthetic HR data saved to: /content/pxt-hr-data-governance/data/raw/hr_employees_raw.csv


,Employee_ID,First_Name,Last_Name,Department,Job_Title,Manager_ID,Location,Country,Employment_Type,Hire_Date,Termination_Date,Status,Base_Salary
0,93810,Rodney,Murphy,Operations,Analyst,50888,New York,USA,Contract,2019-12-24,2022-05-01,Active,59288
1,24592,Jason,Lin,IT,Associate,None,New York,USA,Contract,2022-10-15,None,Terminated,134186
2,13278,Amber,Rasmussen,Marketing,Director,None,Wilmington,USA,PT,2020-08-04,None,Terminated,51540
3,46048,Jessica,Ramirez,IT Dept,Analyst,59192,New York,USA,PT,2016-03-21,2023-08-17,Active,142768
4,42098,James,May,Operations,Analyst,61990,Chicago,USA,FT,2018-10-10,2021-04-05,Active,126958


In [4]:
# Create metadata files
import pandas as pd

base_meta_path = "/content/pxt-hr-data-governance/metadata"

# Business metadata
business_metadata = pd.DataFrame([
    ["HR", "hr_employees_raw", "Employee_ID", "Unique identifier for each employee", None, "Confidential", "PXT Data Steward", ""],
    ["HR", "hr_employees_raw", "Department", "Employee's department", "HR, IT, Finance, Operations, Marketing, Sales", "Internal", "PXT Data Steward", ""],
    ["HR", "hr_employees_raw", "Manager_ID", "ID of employee's manager", None, "Confidential", "PXT Data Steward", ""],
    ["HR", "hr_employees_raw", "Hire_Date", "Date employee was hired", None, "Internal", "PXT Data Steward", ""],
    ["HR", "hr_employees_raw", "Termination_Date", "End of employment", None, "Internal", "PXT Data Steward", ""],
])

business_metadata.columns = [
    "domain", "table_name", "column_name", "business_definition",
    "valid_values", "sensitivity_class", "steward_name", "notes"
]
business_metadata.to_csv(f"{base_meta_path}/business_metadata.csv", index=False)

# Technical metadata
technical_metadata = pd.DataFrame([
    ["hr_employees_raw", "Employee_ID", "int64", False, True, None, "HRIS", "daily", True],
    ["hr_employees_raw", "Department", "string", True, False, None, "HRIS", "daily", False],
    ["hr_employees_raw", "Hire_Date", "date", True, False, None, "HRIS", "daily", False],
    ["hr_employees_raw", "Termination_Date", "date", True, False, None, "HRIS", "daily", False],
])

technical_metadata.columns = [
    "table_name", "column_name", "data_type", "nullable",
    "primary_key", "foreign_key", "source_system", "refresh_frequency",
    "pii_flag"
]
technical_metadata.to_csv(f"{base_meta_path}/technical_metadata.csv", index=False)

# Data quality rules
quality_rules = pd.DataFrame([
    [1, "Employee_ID", "Completeness", "Employee_ID must not be null", "df['Employee_ID'].notnull()", "Critical", 0.99],
    [2, "Employee_ID", "Uniqueness", "Employee ID must be unique", "df['Employee_ID'].duplicated() == False", "High", 0.98],
    [3, "Department", "Validity", "Department must be standardized", "df['Department'].isin(['HR','IT','Finance','Operations','Marketing','Sales'])", "Medium", 0.95],
    [4, "Hire_Date", "Validity", "Hire date cannot be in the future", "pd.to_datetime(df['Hire_Date'], errors='coerce') <= pd.Timestamp.today()", "High", 0.98],
])

quality_rules.columns = [
    "rule_id", "column_name", "dimension", "rule_description",
    "rule_expression", "severity", "target_threshold"
]
quality_rules.to_csv(f"{base_meta_path}/data_quality_rules.csv", index=False)

# Domain owners
domain_owners = pd.DataFrame([
    ["HR", "Employee Master Data", "Jane Doe", "jane.doe@company.com", 0.97, ""],
    ["HR", "Org Hierarchy", "Mark Hayes", "mark.hayes@company.com", 0.95, ""],
])

domain_owners.columns = [
    "domain", "subdomain", "owner_name", "owner_email",
    "sla_data_quality_score", "notes"
]
domain_owners.to_csv(f"{base_meta_path}/domain_owners.csv", index=False)

print("Business, technical, quality rules, and domain owner metadata saved.")

Business, technical, quality rules, and domain owner metadata saved.


In [5]:
# src/__init__.py
%%writefile /content/pxt-hr-data-governance/src/__init__.py
# Make src a package so we can use `python -m src.module_name`

Writing /content/pxt-hr-data-governance/src/__init__.py


In [6]:
# src/config.py
%%writefile /content/pxt-hr-data-governance/src/config.py
from pathlib import Path

# Base project directory for Colab
BASE_DIR = Path("/content/pxt-hr-data-governance")

DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
STAGING_DIR = DATA_DIR / "staging"
CURATED_DIR = DATA_DIR / "curated"

METADATA_DIR = BASE_DIR / "metadata"
DOCS_DIR = BASE_DIR / "docs"

RAW_HR_FILE = RAW_DIR / "hr_employees_raw.csv"
STAGING_HR_FILE = STAGING_DIR / "hr_employees_staging.csv"
CURATED_HR_FILE = CURATED_DIR / "hr_employees_curated.csv"

Writing /content/pxt-hr-data-governance/src/config.py


In [7]:
# src/metadata_loader.py
%%writefile /content/pxt-hr-data-governance/src/metadata_loader.py
import pandas as pd
from .config import METADATA_DIR

def load_business_metadata() -> pd.DataFrame:
    """Load business metadata definitions."""
    return pd.read_csv(METADATA_DIR / "business_metadata.csv")

def load_technical_metadata() -> pd.DataFrame:
    """Load technical metadata for tables/columns."""
    return pd.read_csv(METADATA_DIR / "technical_metadata.csv")

def load_data_quality_rules() -> pd.DataFrame:
    """Load data quality rules configuration."""
    return pd.read_csv(METADATA_DIR / "data_quality_rules.csv")

def load_domain_owners() -> pd.DataFrame:
    """Load domain owners / stewards."""
    return pd.read_csv(METADATA_DIR / "domain_owners.csv")

if __name__ == "__main__":
    print("Business metadata:")
    try:
        print(load_business_metadata().head())
    except FileNotFoundError:
        print("business_metadata.csv not found yet.")


Writing /content/pxt-hr-data-governance/src/metadata_loader.py


In [8]:
# src/data_profiling.py
%%writefile /content/pxt-hr-data-governance/src/data_profiling.py
import pandas as pd
from .config import RAW_HR_FILE, STAGING_DIR
from .metadata_loader import load_business_metadata

def basic_profile(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a simple data profile:
    - dtype
    - non-null count
    - null count
    - distinct count
    """
    profile = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.astype(str).values,
        "non_null_count": df.notnull().sum().values,
        "null_count": df.isnull().sum().values,
        "distinct_count": df.nunique().values
    })
    return profile

def main():
    # Load raw HR data
    df_raw = pd.read_csv(RAW_HR_FILE)

    # Basic profile
    profile = basic_profile(df_raw)

    # Ensure staging dir exists
    STAGING_DIR.mkdir(parents=True, exist_ok=True)
    profile.to_csv(STAGING_DIR / "hr_profile_raw.csv", index=False)

    print("=== Raw HR Data Profile ===")
    print(profile)

    # Optional: enrich with business metadata
    try:
        business_meta = load_business_metadata()
        merged = profile.merge(
            business_meta,
            left_on="column",
            right_on="column_name",
            how="left"
        )
        merged.to_csv(STAGING_DIR / "hr_profile_with_metadata.csv", index=False)
        print("\n=== Profile joined with business metadata ===")
        print(merged.head())
    except FileNotFoundError:
        print("\nBusiness metadata not found, skipping enriched profile.")

if __name__ == "__main__":
    main()


Writing /content/pxt-hr-data-governance/src/data_profiling.py


In [9]:
# src/data_quality_checks.py
%%writefile /content/pxt-hr-data-governance/src/data_quality_checks.py
import pandas as pd
from .config import RAW_HR_FILE, STAGING_DIR
from .metadata_loader import load_data_quality_rules

def run_rule(df: pd.DataFrame, rule_row: pd.Series) -> float:
    """
    Evaluate a simple Python-based rule expression against the dataframe.

    rule_expression should be a boolean Series expression using 'df'.
    Example: 'df[\"Employee_ID\"].notnull()'

    Returns:
        Passing ratio as a float between 0 and 1.
    """
    expr = rule_row["rule_expression"]
    # NOTE: In a real production system you would parse this safely instead of eval.
    passed = eval(expr)
    return float(passed.mean())

def main():
    df = pd.read_csv(RAW_HR_FILE)
    rules = load_data_quality_rules()

    results = []
    for _, row in rules.iterrows():
        score = run_rule(df, row)
        results.append({
            "rule_id": row["rule_id"],
            "column_name": row["column_name"],
            "dimension": row["dimension"],
            "severity": row["severity"],
            "target_threshold": row["target_threshold"],
            "score": score
        })

    results_df = pd.DataFrame(results)

    STAGING_DIR.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(STAGING_DIR / "hr_data_quality_results_raw.csv", index=False)

    print("=== Data Quality Results (Raw) ===")
    print(results_df)

if __name__ == "__main__":
    main()


Writing /content/pxt-hr-data-governance/src/data_quality_checks.py


In [10]:
# src/data_cleansing.py
%%writefile /content/pxt-hr-data-governance/src/data_cleansing.py
import pandas as pd
from .config import (
    RAW_HR_FILE,
    STAGING_HR_FILE,
    CURATED_HR_FILE,
    STAGING_DIR,
    CURATED_DIR,
)

def clean_departments(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize department labels:
    - 'Human Resources', 'H.R.', 'Hr' -> 'HR'
    - 'Information Technology', 'IT Dept' -> 'IT'
    """
    mapping = {
        "Human Resources": "HR",
        "H.R.": "HR",
        "Hr": "HR",
        "Information Technology": "IT",
        "IT Dept": "IT",
    }
    if "Department" in df.columns:
        df["Department"] = df["Department"].replace(mapping)
    return df

def fix_dates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Parse dates and fix impossible values such as:
    - Termination date before hire date -> set Termination_Date to NaT
    """
    if "Hire_Date" in df.columns:
        df["Hire_Date"] = pd.to_datetime(df["Hire_Date"], errors="coerce")
    if "Termination_Date" in df.columns:
        df["Termination_Date"] = pd.to_datetime(df["Termination_Date"], errors="coerce")
        # Ensure no Termination_Date before Hire_Date
        if "Hire_Date" in df.columns:
            mask = df["Termination_Date"] < df["Hire_Date"]
            df.loc[mask, "Termination_Date"] = pd.NaT
    return df

def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """Drop duplicate employee rows based on Employee_ID."""
    if "Employee_ID" in df.columns:
        return df.drop_duplicates(subset=["Employee_ID"])
    return df.drop_duplicates()

def main():
    df_raw = pd.read_csv(RAW_HR_FILE)

    df_stage = df_raw.copy()
    df_stage = clean_departments(df_stage)
    df_stage = fix_dates(df_stage)
    df_stage = drop_duplicates(df_stage)

    STAGING_DIR.mkdir(parents=True, exist_ok=True)
    CURATED_DIR.mkdir(parents=True, exist_ok=True)

    df_stage.to_csv(STAGING_HR_FILE, index=False)
    df_stage.to_csv(CURATED_HR_FILE, index=False)

    print("Saved staging and curated HR data:")
    print(f" - Staging: {STAGING_HR_FILE}")
    print(f" - Curated: {CURATED_HR_FILE}")
    print("\nSample of curated data:")
    print(df_stage.head())

if __name__ == "__main__":
    main()


Writing /content/pxt-hr-data-governance/src/data_cleansing.py


In [11]:
# src/lineage_builder.py
%%writefile /content/pxt-hr-data-governance/src/lineage_builder.py
import pandas as pd
from .config import DOCS_DIR

def build_lineage() -> pd.DataFrame:
    """
    Build a simple lineage mapping from raw to curated.

    Columns:
        source_table, source_column, target_table, target_column, transformation
    """
    lineage_rows = [
        [
            "hr_employees_raw",
            "Employee_ID",
            "hr_employees_curated",
            "Employee_ID",
            "deduplicated on Employee_ID",
        ],
        [
            "hr_employees_raw",
            "Department",
            "hr_employees_curated",
            "Department",
            "standardized labels (HR/IT/etc.)",
        ],
        [
            "hr_employees_raw",
            "Hire_Date",
            "hr_employees_curated",
            "Hire_Date",
            "parsed to datetime, invalid reset to NaT",
        ],
        [
            "hr_employees_raw",
            "Termination_Date",
            "hr_employees_curated",
            "Termination_Date",
            "parsed to datetime, invalid/future/early fixed or set to NaT",
        ],
    ]
    df = pd.DataFrame(
        lineage_rows,
        columns=[
            "source_table",
            "source_column",
            "target_table",
            "target_column",
            "transformation",
        ],
    )
    return df

def main():
    DOCS_DIR.mkdir(parents=True, exist_ok=True)
    lineage_df = build_lineage()
    out_path = DOCS_DIR / "hr_lineage_table.csv"
    lineage_df.to_csv(out_path, index=False)

    print("=== HR Lineage Table ===")
    print(lineage_df)
    print(f"\nSaved lineage table to: {out_path}")

if __name__ == "__main__":
    main()


Writing /content/pxt-hr-data-governance/src/lineage_builder.py


In [12]:
# Simple Streamlit metadata browser
%%writefile /content/pxt-hr-data-governance/dashboard/streamlit_app.py
import streamlit as st
import pandas as pd
from pathlib import Path

BASE_DIR = Path("/content/pxt-hr-data-governance")
METADATA_DIR = BASE_DIR / "metadata"
STAGING_DIR = BASE_DIR / "data" / "staging"

st.title("HR Data Catalog & Quality Overview")

@st.cache_data
def load_metadata():
    business = pd.read_csv(METADATA_DIR / "business_metadata.csv")
    technical = pd.read_csv(METADATA_DIR / "technical_metadata.csv")
    quality = pd.read_csv(STAGING_DIR / "hr_data_quality_results_raw.csv")
    return business, technical, quality

business, technical, quality = load_metadata()

search_term = st.text_input("Search by column name:", "")

if search_term:
    biz_filtered = business[business["column_name"].str.contains(search_term, case=False, na=False)]
    tech_filtered = technical[technical["column_name"].str.contains(search_term, case=False, na=False)]
    qual_filtered = quality[quality["column_name"].str.contains(search_term, case=False, na=False)]

    st.subheader("Business Metadata")
    st.dataframe(biz_filtered)

    st.subheader("Technical Metadata")
    st.dataframe(tech_filtered)

    st.subheader("Data Quality Rules / Scores")
    st.dataframe(qual_filtered)
else:
    st.write("Type a column name (e.g., Employee_ID, Department) to explore metadata and quality scores.")


Writing /content/pxt-hr-data-governance/dashboard/streamlit_app.py


In [13]:
# Run for UI in Web
!streamlit run /content/pxt-hr-data-governance/dashboard/streamlit_app.py --server.port 8501 --server.address 0.0.0.0





  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

  Stopping...
  Stopping...


In [15]:
# Run the pipeline (profiling → quality → cleansing → lineage)

# Profile raw data
!cd /content/pxt-hr-data-governance && python -m src.data_profiling

# Run data quality rules
!cd /content/pxt-hr-data-governance && python -m src.data_quality_checks

# Cleanse and create curated data
!cd /content/pxt-hr-data-governance && python -m src.data_cleansing

# Generate lineage table
!cd /content/pxt-hr-data-governance && python -m src.lineage_builder



=== Raw HR Data Profile ===
              column    dtype  non_null_count  null_count  distinct_count
0        Employee_ID    int64             205           0             200
1         First_Name   object             205           0             124
2          Last_Name   object             205           0             158
3         Department   object             205           0              10
4          Job_Title   object             205           0               6
5         Manager_ID  float64             155          50             152
6           Location   object             205           0               5
7            Country   object             205           0               1
8    Employment_Type   object             205           0               3
9          Hire_Date   object             205           0             192
10  Termination_Date   object              26         179              25
11            Status   object             205           0               2
12       B

In [16]:
# Quick peek at outputs

import pandas as pd

profile = pd.read_csv("/content/pxt-hr-data-governance/data/staging/hr_profile_raw.csv")
dq_results = pd.read_csv("/content/pxt-hr-data-governance/data/staging/hr_data_quality_results_raw.csv")
curated = pd.read_csv("/content/pxt-hr-data-governance/data/curated/hr_employees_curated.csv")
lineage = pd.read_csv("/content/pxt-hr-data-governance/docs/hr_lineage_table.csv")

print("=== Profile (first 5 rows) ===")
display(profile.head())

print("=== Data Quality Results ===")
display(dq_results)

print("=== Curated HR Data (first 5 rows) ===")
display(curated.head())

print("=== Lineage Table ===")
display(lineage)


=== Profile (first 5 rows) ===


,column,dtype,non_null_count,null_count,distinct_count
0,Employee_ID,int64,205,0,200
1,First_Name,object,205,0,124
2,Last_Name,object,205,0,158
3,Department,object,205,0,10
4,Job_Title,object,205,0,6


=== Data Quality Results ===


,rule_id,column_name,dimension,severity,target_threshold,score
0,1,Employee_ID,Completeness,Critical,0.99,1.000000
1,2,Employee_ID,Uniqueness,High,0.98,0.975610
2,3,Department,Validity,Medium,0.95,0.687805
3,4,Hire_Date,Validity,High,0.98,0.985366


=== Curated HR Data (first 5 rows) ===


,Employee_ID,First_Name,Last_Name,Department,Job_Title,Manager_ID,Location,Country,Employment_Type,Hire_Date,Termination_Date,Status,Base_Salary
0,93810,Rodney,Murphy,Operations,Analyst,50888.0,New York,USA,Contract,2019-12-24,2022-05-01,Active,59288
1,24592,Jason,Lin,IT,Associate,NaN,New York,USA,Contract,2022-10-15,NaN,Terminated,134186
2,13278,Amber,Rasmussen,Marketing,Director,NaN,Wilmington,USA,PT,2020-08-04,NaN,Terminated,51540
3,46048,Jessica,Ramirez,IT,Analyst,59192.0,New York,USA,PT,2016-03-21,2023-08-17,Active,142768
4,42098,James,May,Operations,Analyst,61990.0,Chicago,USA,FT,2018-10-10,2021-04-05,Active,126958


=== Lineage Table ===


,source_table,source_column,target_table,target_column,transformation
0,hr_employees_raw,Employee_ID,hr_employees_curated,Employee_ID,deduplicated on Employee_ID
1,hr_employees_raw,Department,hr_employees_curated,Department,standardized labels (HR/IT/etc.)
2,hr_employees_raw,Hire_Date,hr_employees_curated,Hire_Date,"parsed to datetime, invalid reset to NaT"
3,hr_employees_raw,Termination_Date,hr_employees_curated,Termination_Date,"parsed to datetime, invalid/future/early fixed..."
